# ORM Concepts

## Learning Objectives
- Understand Object-Relational Mapping (ORM) concepts
- Learn how SQLAlchemy works as an ORM
- Master the difference between Core and ORM approaches
- Understand database engines and connections
- Learn about declarative base and model creation

## What You'll Learn
- What ORMs are and why we use them
- SQLAlchemy architecture and components
- Database engines and connection strings
- Declarative base and model definitions
- The difference between SQLAlchemy Core and ORM
- Basic setup and configuration


## What is an ORM?

Object-Relational Mapping (ORM) is a programming technique that allows you to work with databases using object-oriented programming concepts. Instead of writing raw SQL queries, you work with Python objects that represent database tables and rows.


## Why Use an ORM?

### Benefits:
- **Productivity**: Write less boilerplate code
- **Type Safety**: Catch errors at development time
- **Database Agnostic**: Switch databases without changing code
- **Security**: Built-in protection against SQL injection
- **Maintainability**: Easier to read and maintain code

### Example Comparison:

**Raw SQL:**
```sql
SELECT * FROM users WHERE age > 18 AND city = 'New York';
```

**ORM (SQLAlchemy):**
```python
session.query(User).filter(User.age > 18, User.city == 'New York').all()
```


## SQLAlchemy Architecture

SQLAlchemy has two main components:

### 1. SQLAlchemy Core
- **Low-level**: Direct SQL expression language
- **Database agnostic**: Works with any SQL database
- **Performance**: More control over generated SQL

### 2. SQLAlchemy ORM
- **High-level**: Object-oriented interface
- **Pythonic**: Works with Python classes and objects
- **Productivity**: Faster development with less code

### Key Components:
- **Engine**: Manages database connections
- **Session**: Manages object state and transactions
- **Models**: Python classes representing database tables
- **Query**: Object-oriented query interface


## 1. Setting Up SQLAlchemy

Let's start with the basic setup:


In [ ]:
# Install SQLAlchemy (if not already installed)
# pip install sqlalchemy

# Basic imports
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime

# Create the base class for our models
Base = declarative_base()

print("SQLAlchemy setup complete!")


## 2. Database Engine and Connection

The engine is the starting point for any SQLAlchemy application:


In [ ]:
# Create a database engine
# For this example, we'll use SQLite (file-based database)
engine = create_engine('sqlite:///example.db', echo=True)

# echo=True shows the SQL queries being executed
# This is helpful for learning and debugging

# Create a session factory
Session = sessionmaker(bind=engine)

# Create a session
session = Session()

print("Database engine and session created!")
print(f"Engine: {engine}")
print(f"Session: {session}")


## 3. Creating Your First Model

Models are Python classes that represent database tables:


In [ ]:
# Define a User model
class User(Base):
    __tablename__ = 'users'
    
    # Primary key
    id = Column(Integer, primary_key=True)
    
    # User attributes
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    def __repr__(self):
        return f"<User(username='{self.username}', email='{self.email}')>"

print("User model defined!")
print(f"Table name: {User.__tablename__}")
print(f"Columns: {User.__table__.columns.keys()}")


## 4. Creating Tables

Now let's create the actual database tables:


In [ ]:
# Create all tables defined in our models
Base.metadata.create_all(engine)

print("Tables created successfully!")
print("Check your directory - you should see 'example.db' file")


## 5. Basic CRUD Operations

Let's perform Create, Read, Update, Delete operations:


In [ ]:
# CREATE - Add new users
user1 = User(username='alice', email='alice@example.com')
user2 = User(username='bob', email='bob@example.com')

# Add to session
session.add(user1)
session.add(user2)

# Commit the transaction
session.commit()

print("Users created successfully!")
print(f"User 1: {user1}")
print(f"User 2: {user2}")


In [ ]:
# READ - Query users
# Get all users
all_users = session.query(User).all()
print("All users:")
for user in all_users:
    print(f"  {user}")

# Get user by ID
user_by_id = session.query(User).filter(User.id == 1).first()
print(f"\nUser with ID 1: {user_by_id}")

# Get user by username
user_by_username = session.query(User).filter(User.username == 'alice').first()
print(f"User with username 'alice': {user_by_username}")


In [ ]:
# UPDATE - Modify existing user
user_to_update = session.query(User).filter(User.username == 'alice').first()
if user_to_update:
    user_to_update.email = 'alice.newemail@example.com'
    session.commit()
    print(f"Updated user: {user_to_update}")
else:
    print("User not found")


In [ ]:
# DELETE - Remove a user
user_to_delete = session.query(User).filter(User.username == 'bob').first()
if user_to_delete:
    session.delete(user_to_delete)
    session.commit()
    print("User 'bob' deleted successfully")
else:
    print("User not found")

# Verify deletion
remaining_users = session.query(User).all()
print(f"\nRemaining users: {len(remaining_users)}")
for user in remaining_users:
    print(f"  {user}")


## 6. Session Management

Proper session management is crucial for database operations:


In [ ]:
# Always close the session when done
session.close()

# Using context manager for automatic session management
def create_user_with_context(username, email):
    """Create a user using context manager for session"""
    with Session() as session:
        user = User(username=username, email=email)
        session.add(user)
        session.commit()
        return user

# Test the context manager approach
new_user = create_user_with_context('charlie', 'charlie@example.com')
print(f"Created user with context manager: {new_user}")

# Verify the user was created
with Session() as session:
    all_users = session.query(User).all()
    print(f"\nTotal users in database: {len(all_users)}")
    for user in all_users:
        print(f"  {user}")


## 7. Common Column Types

SQLAlchemy provides many column types for different data:


In [ ]:
from sqlalchemy import Boolean, Float, Text, Date, Time

# Example model with various column types
class Product(Base):
    __tablename__ = 'products'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    description = Column(Text)  # For long text
    price = Column(Float, nullable=False)
    is_active = Column(Boolean, default=True)
    created_date = Column(Date, default=datetime.utcnow().date())
    created_time = Column(Time, default=datetime.utcnow().time())
    
    def __repr__(self):
        return f"<Product(name='{self.name}', price={self.price})>"

print("Product model with various column types defined!")
print("Column types used:")
print("- Integer: for IDs and numbers")
print("- String: for short text with length limit")
print("- Text: for long text without length limit")
print("- Float: for decimal numbers")
print("- Boolean: for true/false values")
print("- Date: for dates only")
print("- Time: for time only")
print("- DateTime: for date and time together")


## Summary

You've learned about:

1. **ORM Concepts** - Object-Relational Mapping fundamentals
2. **SQLAlchemy Setup** - Installing and configuring SQLAlchemy
3. **Database Engine** - Creating connections to databases
4. **Models** - Defining Python classes for database tables
5. **CRUD Operations** - Create, Read, Update, Delete operations
6. **Session Management** - Proper handling of database sessions
7. **Column Types** - Different data types for various use cases

## Key Takeaways

- ORMs make database operations more Pythonic and secure
- SQLAlchemy provides both Core and ORM approaches
- Always use proper session management
- Models represent database tables as Python classes
- CRUD operations are the foundation of database interactions
- Choose appropriate column types for your data

Ready to practice? Move on to the exercise notebook!
